In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
from config import conn_string

**read all files**

In [2]:
# path
station_path = os.path.join('.', "station_info.csv")
weather_path = os.path.join('.', "historical_weather.csv")

#bikes
toronto_bikes = os.path.join("Resources/1_toronto_canada", "toronto_bikes.csv")
vancouver_bikes = os.path.join("Resources/2_vancouver_canada", "vancouver_bikes.csv")
boston_bikes = os.path.join("Resources/3_boston_us", "boston_bikes.csv")
nyc_bikes = os.path.join("Resources/4_NewYork_US", "nyc_bikes.csv")

# membertypes
toronto_users = os.path.join("Resources/1_toronto_canada", "toronto_membertypes.csv")
vancouver_users = os.path.join("Resources/2_vancouver_canada", "vancouver_membertypes.csv")
boston_users = os.path.join("Resources/3_boston_us", "boston_membertypes.csv")
nyc_users = os.path.join("Resources/4_NewYork_US", "nyc_membertypes.csv")

# ridership
toronto_ = os.path.join("Resources/1_toronto_canada", "toronto_bikeshare.csv")
vancouver_ = os.path.join("Resources/2_vancouver_canada", "vancouver_bikeshare.csv")
boston_ = os.path.join("Resources/3_boston_us", "boston_bikeshare.csv")
nyc_ = os.path.join("Resources/4_NewYork_US", "nyc_bikeshare.csv")


In [3]:
# read files
stns = pd.read_csv(station_path, encoding="utf-8")
weather = pd.read_csv(weather_path, encoding="utf-8")

# bikes
tor_bikes = pd.read_csv(toronto_bikes, encoding="utf-8")
van_bikes = pd.read_csv(vancouver_bikes, encoding="utf-8")
bos_bikes = pd.read_csv(boston_bikes, encoding="utf-8")
nyc_bikes = pd.read_csv(nyc_bikes, encoding="utf-8")

# member_types
tor_users = pd.read_csv(toronto_users, encoding="utf-8")
van_users = pd.read_csv(vancouver_users, encoding="utf-8")
bos_users = pd.read_csv(boston_users, encoding="utf-8")
nyc_users = pd.read_csv(nyc_users, encoding="utf-8")

# pricing index


# ridership
toronto = pd.read_csv(toronto_, encoding="utf-8")
vancouver = pd.read_csv(vancouver_, encoding="utf-8")
boston = pd.read_csv(boston_, encoding="utf-8")
newyork = pd.read_csv(nyc_, encoding="utf-8")


In [42]:
toronto = pd.read_csv(toronto_, encoding="utf-8")

In [43]:
toronto.count()

trip_duration       2439063
start_date          2439063
end_date            2439063
start_station_id    2439063
end_station_id      2439063
bike_id             2439063
member_type         2439063
location_id         2439063
dtype: int64

**Locations**

In [6]:
locations = [{"location_id": 1,"city":"Toronto", "country":"Canada" }, 
             {"location_id": 2,"city":"Vancouver", "country":"Canada"},
             {"location_id": 3,"city":"Boston", "country":"USA"},
             {"location_id": 4,"city":"New York", "country":"USA"}
            ]

df = pd.DataFrame(locations)
df.set_index("location_id", inplace=True)
df


,city,country
location_id,,
1,Toronto,Canada
2,Vancouver,Canada
3,Boston,USA
4,New York,USA


**Stations**

In [7]:
stns.head()

,station_id,station_name,latitude,longitude,location_id
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,1
1,7001,Lower Jarvis St / The Esplanade,43.647830,-79.370698,1
2,7002,St. George St / Bloor St W,43.667333,-79.399429,1
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761,1
4,7004,University Ave / Elm St,43.656518,-79.389099,1


**Historical Weather**

In [8]:
weather.head()

,location_id,forecast_date,maxtempc,humidity,total_precip,avg_cloudcover,avg_windspeed
0,1,2019-01-01,1,77.250,3.6,50.625,16.875
1,1,2019-01-02,-5,69.000,0.2,41.750,8.625
2,1,2019-01-03,-1,68.500,1.3,68.250,17.750
3,1,2019-01-04,2,77.250,0.0,7.625,18.125
4,1,2019-01-05,3,81.125,0.0,23.750,11.250


**Bikes**

In [9]:
tor_bikes.head()

,bike_id,location_id
0,1296,1
1,2947,1
2,2293,1
3,283,1
4,1799,1


In [10]:
van_bikes.head()

,bike_id,location_id
0,388.0,2
1,1880.0,2
2,779.0,2
3,2123.0,2
4,444.0,2


In [11]:
bos_bikes.head()

,bike_id,location_id
0,3689,3
1,4142,3
2,1628,3
3,2969,3
4,3469,3


In [12]:
nyc_bikes.head()

,bike_id,location_id
0,15839,4
1,32723,4
2,27451,4
3,21579,4
4,35379,4


**Member Types**

In [13]:
tor_users.head()

,member_type,location_id
0,Annual Member,1
1,Casual Member,1


In [14]:
van_users.head()

,member_type,location_id
0,365 Standard,2
1,365 Plus,2
2,Vancity Community Pass,2
3,365 Day Pass Plus SALE,2
4,365 Day Founding Standard,2


In [15]:
bos_users.head()

,member_type,location_id
0,Subscriber,3
1,Customer,3


In [17]:
nyc_users.head()

,member_type,location_id
0,Subscriber,4
1,Customer,4


**Pricing Index**

**Ridership**

In [18]:
# Toronto

toronto.head()

,trip_duration,start_date,end_date,start_station_id,end_station_id,bike_id,member_type,location_id
0,1547,01/01/2019 00:08,01/01/2019 00:33,7021,7233.0,1296,Annual Member,1
1,1112,01/01/2019 00:10,01/01/2019 00:29,7160,7051.0,2947,Annual Member,1
2,589,01/01/2019 00:15,01/01/2019 00:25,7055,7013.0,2293,Annual Member,1
3,259,01/01/2019 00:16,01/01/2019 00:20,7012,7235.0,283,Annual Member,1
4,281,01/01/2019 00:19,01/01/2019 00:24,7041,7257.0,1799,Annual Member,1


### Load data

In [19]:
cnx = f'postgresql://{conn_string}'
engine = create_engine(cnx)

In [20]:
# confirm tables
engine.table_names()

['locations',
 'membertypes',
 'stations',
 'bikes',
 'ridership',
 'pricing',
 'historicalweather']

In [ ]:
# load locations into database
df.to_sql(name='locations', con=engine, if_exists='append', index=True)

In [ ]:
# load stations into database
stns.to_sql(name='stations', con=engine, if_exists='append', index=False)


In [ ]:
# load historical weather into database
weather.to_sql(name='historicalweather', con=engine, if_exists='append', index=False)

In [ ]:
# load toronto bikes into database
tor_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)

In [ ]:
# load vancouver bikes into database
van_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)

In [ ]:
# load boston bikes into database
bos_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)

In [ ]:
# load new york bikes into database
nyc_bikes.to_sql(name='bikes', con=engine, if_exists='append', index=False)

In [ ]:
# load toronto member types
tor_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)

In [ ]:
# load vancouver member types
van_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)

In [ ]:
# load boston member types
bos_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)

In [ ]:
# load new york member types
nyc_users.to_sql(name='membertypes', con=engine, if_exists='append', index=False)

**Load Ridership**

In [21]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

# check reference to stations table
# create session
session = Session(bind=engine)
# declare a base using automap_base
Base = automap_base()
# use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)
Base.classes.keys()
Stations = Base.classes.stations

*Verify that all station_ids used in bikeshare exists in Stations table*

In [22]:
all_stations = session.query(Stations.station_id).all()
all_stns_df = pd.DataFrame(all_stations)
all_stns_df

,station_id
0,7000
1,7001
2,7002
3,7003
4,7004
...,...
2396,4248
2397,4249
2398,4250
2399,4252


***Toronto***

In [44]:
missing_start = toronto[-toronto["start_station_id"].isin(all_stns_df["station_id"])]
missing_start = missing_start.rename(columns={"start_station_id":"station_id"})


In [45]:
missing_start

,trip_duration,start_date,end_date,station_id,end_station_id,bike_id,member_type,location_id
666892,20,06/14/2019 15:49,06/14/2019 15:49,7393,7219.0,242,Annual Member,1
666906,111,06/14/2019 15:50,06/14/2019 15:52,7393,7219.0,242,Annual Member,1
666944,16,06/14/2019 15:52,06/14/2019 15:53,7393,7219.0,242,Annual Member,1
732424,20,06/21/2019 11:06,06/21/2019 11:06,7219,7124.0,3374,Annual Member,1
732452,16,06/21/2019 11:10,06/21/2019 11:10,7219,7124.0,2267,Annual Member,1
...,...,...,...,...,...,...,...,...
2436793,489,12/30/2019 19:46,12/30/2019 19:54,7416,7372.0,2563,Annual Member,1
2437225,231,12/31/2019 06:41,12/31/2019 06:45,7416,7010.0,1352,Annual Member,1
2437498,754,12/31/2019 09:12,12/31/2019 09:24,7416,7248.0,1799,Annual Member,1
2438159,394,12/31/2019 14:41,12/31/2019 14:48,7416,7075.0,4520,Annual Member,1


In [46]:
missing_end = toronto[-toronto["end_station_id"].isin(all_stns_df["station_id"])]
missing_end = missing_end.rename(columns={"start_station_id":"station_id"})


In [47]:
missing_end

,trip_duration,start_date,end_date,station_id,end_station_id,bike_id,member_type,location_id
465037,640,05/22/2019 06:37,05/22/2019 06:47,7163,7394.0,3317,Annual Member,1
472411,926,05/23/2019 06:29,05/23/2019 06:45,7163,7394.0,3104,Annual Member,1
474540,3647,05/23/2019 10:29,05/23/2019 11:30,7023,7394.0,1121,Casual Member,1
483978,336332,05/24/2019 12:26,05/28/2019 09:51,7051,7394.0,3302,Annual Member,1
656208,493,06/13/2019 06:43,06/13/2019 06:51,7055,7394.0,4366,Annual Member,1
...,...,...,...,...,...,...,...,...
2435173,311,12/30/2019 12:44,12/30/2019 12:49,7411,7416.0,1799,Annual Member,1
2436190,262,12/30/2019 17:05,12/30/2019 17:10,7057,7416.0,1352,Annual Member,1
2436295,426,12/30/2019 17:25,12/30/2019 17:32,7382,7416.0,2563,Annual Member,1
2436586,338,12/30/2019 18:30,12/30/2019 18:36,7366,7416.0,2707,Annual Member,1


**Load rows with station_id existing in stations table**

In [38]:
toronto = toronto[toronto["start_station_id"].isin(all_stns_df["station_id"])]
toronto = toronto[toronto["end_station_id"].isin(all_stns_df["station_id"])]

In [39]:
toronto.to_sql(name='ridership', con=engine, if_exists='append', index=False, index_label='id')

***Vancouver***

In [48]:
missing_start = vancouver[-vancouver["start_station_id"].isin(all_stns_df["station_id"])]
missing_start = missing_start.rename(columns={"start_station_id":"station_id"})


In [49]:
missing_start

,trip_duration,start_date,end_date,station_id,end_station_id,bike_id,member_type,location_id
8712,28,1/25/2019 18:00,1/25/2019 18:00,990,990,1306.0,VIP,2
9630,170,1/25/2019 9:00,1/25/2019 9:00,990,225,1412.0,VIP,2
11715,30,1/23/2019 17:00,1/23/2019 17:00,991,990,681.0,VIP,2
12212,1600,1/23/2019 11:00,1/23/2019 11:00,990,54,412.0,VIP,2
12238,34,1/23/2019 10:00,1/23/2019 10:00,990,990,412.0,VIP,2
...,...,...,...,...,...,...,...,...
889372,5032,2019-12-02 18:00,2019-12-02 19:00,981,72,1428.0,VIP,2
889441,1042,2019-12-02 17:00,2019-12-02 17:00,981,52,1761.0,VIP,2
889506,771,2019-12-02 16:00,2019-12-02 16:00,981,12,150.0,VIP,2
889602,2059,2019-12-02 14:00,2019-12-02 14:00,981,981,1647.0,VIP,2
